In [2]:
import pandas as pd
from pandas.tseries.offsets import BDay

isBusinessDay = BDay().onOffset
csv_path = 'Datasets/ETH2.csv'
dates_df = pd.read_csv(csv_path)
match_series = pd.to_datetime(dates_df['Date']).map(isBusinessDay)
dates_df[match_series]

/Users/sareddyrojareddy/opt/anaconda3/lib/python3.9/site-packages/pandas/core/base.py:870: FutureWarning: onOffset is a deprecated, use is_on_offset instead
  new_values = map_f(values, mapper)


,Date,Price,Open,High,Low,Vol.,Change %
0,1-Mar-22,"2,949.28","2,922.44","3,036.59","2,858.54",917.00K,0.92%
1,28-Feb-22,"2,922.50","2,616.34","2,949.44","2,574.12",899.19K,11.70%
4,25-Feb-22,"2,767.53","2,596.52","2,830.31","2,573.39",828.87K,6.59%
5,24-Feb-22,"2,596.52","2,578.54","2,732.02","2,303.07",1.85M,0.71%
6,23-Feb-22,"2,578.15","2,637.38","2,749.80","2,578.15",636.50K,-2.20%
...,...,...,...,...,...,...,...
2176,16-Mar-16,12.88,13.06,13.89,12.62,19.24K,-1.38%
2177,15-Mar-16,13.06,12.63,13.42,11.98,39.73K,4.48%
2178,14-Mar-16,12.5,15.07,15.07,11.4,92.18K,-17.05%
2181,11-Mar-16,11.95,11.75,11.95,11.75,0.18K,1.70%
